# Natural Statistics Cross-linguistic: 

#### Lexical diversity analysis -  maximum vocal turns sample

----

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "data_proc")
import analytic_proc
import warnings
warnings.filterwarnings('ignore')

In [2]:
maxturn_dat_inc = pd.read_csv("../data/maxturn_dat_inc_master.csv",index_col=0,low_memory=False)
maxturn_dat_inc=maxturn_dat_inc[maxturn_dat_inc["language"]!="ara"]
maxturn_dat_inc=maxturn_dat_inc[(maxturn_dat_inc["target_child_age"]>=5) & (maxturn_dat_inc["target_child_age"]<=30)]
maxturn_dat_inc_cg = maxturn_dat_inc[maxturn_dat_inc["caregiver"]=="caregiver"]

maxturn_dat_inc_cg["contingent"] = np.where(maxturn_dat_inc_cg["contingent"]==1, "contingent", "non-contingent")
maxturn_dat_inc_cg["swu"]=np.where(maxturn_dat_inc_cg["num_tokens"]==1,1,0)

maxturn_dat_inc_cg = maxturn_dat_inc_cg[maxturn_dat_inc_cg["gloss"].notna()]
maxturn_dat_inc_cg = maxturn_dat_inc_cg[maxturn_dat_inc_cg["gloss"]!="xxx"]
maxturn_dat_inc_cg = maxturn_dat_inc_cg[maxturn_dat_inc_cg["gloss"]!="yyy"]
maxturn_dat_inc_cg = maxturn_dat_inc_cg[maxturn_dat_inc_cg["gloss"]!="www"]

Would call the following if we could run lexical diversity analysis globally:

```python
analytic_proc.create_result(maxturn_dat_inc_cg)
```

However, we want to have seperate dictionaries for contingent and non-contingent words so we can compare them to one another.

The function will allow us to have a different dictionary for each transcript.

Finally, to compare, we can run mixed effects to understand whether contingent and non-contingent utterances differ in their lexical diversity, controlling for number of transcripts.

----
#### Seperate contingent and non-contingent utterances into individual dataframes

In [3]:
maxturn_dat_inc_cg_cc = maxturn_dat_inc_cg[maxturn_dat_inc_cg["contingent"]=="contingent"].reset_index(drop=True)
maxturn_dat_inc_cg_nc = maxturn_dat_inc_cg[maxturn_dat_inc_cg["contingent"]=="non-contingent"].reset_index(drop=True)

----
#### Loop through each unique transcript to compute the lexical diversity counts across languages.

In [5]:
# analytic_proc.create_result(maxturn_dat_inc_cg_cc)

In [4]:
# change the "to_csv" line at the end of analytic_proc.py first

# analytic_proc.create_result(maxturn_dat_inc_cg_nc)

----
#### Lexical Diversity plot

In [4]:
maxturn_dat_inc_cg_cc = pd.read_csv("../data/maxturn_dat_inc_master_cc_lexdiv.csv",index_col=0,low_memory=False)
maxturn_dat_inc_cg_nc = pd.read_csv("../data/maxturn_dat_inc_master_nc_lexdiv.csv",index_col=0,low_memory=False)

# combine dataframes into one

maxturn_dat_inc_cg = pd.concat([maxturn_dat_inc_cg_cc,maxturn_dat_inc_cg_nc])

In [5]:
maxturn_lex_stats = (maxturn_dat_inc_cg.groupby(["Language_name","target_child_id","transcript_id","contingent"])
                                  .uniquenss
                                  .agg(["sum"])
                                  .reset_index())
maxturn_lex_stats =  maxturn_lex_stats.rename({'sum': 'sums'}, axis=1)

In [6]:
len(maxturn_lex_stats["Language_name"].unique())

14

In [7]:
%load_ext rpy2.ipython

Simple plot

In [9]:
%%R -i maxturn_lex_stats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="English")
est_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Norwegian")
pol_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Portuguese")
spa_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Spanish")
swe_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Swedish")
zho_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Mandarin")
# zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(maxturn_lex_stats, aes(x = contingent, y = sums, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=8,color="black") + 
     geom_text(data = eng_label,label = "***",size=8,color="black") +  
     geom_text(data = est_label,label = "**",size=8,color="black") +  
#      geom_text(data = fas_ns_label,label = "*",size=4,color="black",fontface = "italic") +
     geom_text(data = fas_label,label = "*",size=8, color="black") +
     geom_text(data = fra_label,label = "***",size=8,color="black") +  
     geom_text(data = hrv_label,label = "***",size=8,color="black") + 
     geom_text(data = jpn_label,label = "***",size=8,color="black") + 
     geom_text(data = kor_label,label = "***",size=8,color="black") +  
     geom_text(data = nor_ns_label,label = "ns",size=4,color="black",fontface = "italic") +  
#      geom_text(data = pol_ns_label,label = "ns", size=4,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "*",size=8,color="black") +  
     geom_text(data = spa_label,label = "*",size=8,color="black") + 
     geom_text(data = swe_ns_label,label = "ns",size=4,color="black",fontface = "italic") + 
#      geom_text(data = zho_ns_label,label = "ns",size=4,color="black",fontface = "italic") +
     geom_text(data = zho_label,label = "**",size=8, color="black") +
     ylim(0, 250) +
     labs(y = "Lexical Diversity", x = "") +
     theme_classic() +
     theme(text = element_text(size=16),
           axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
     ggsave("../figures/lexical_diversity_maxturn.pdf", width = 11.7, height = 6.2)

Plot + effect estimates

In [41]:
%%R

deu_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="German")
eng_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="English")
est_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Estonian")
fas_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Persian")
fra_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="French")
hrv_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Croatian")
jpn_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Japanese")
kor_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Korean")
nor_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Norwegian")
por_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Portuguese")
spa_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Spanish")
swe_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Swedish")
zho_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Mandarin")

p <- p + geom_text(data = deu_est_label,label = "est=-46.7",size=4,color="black") +
         geom_text(data = eng_est_label,label = "est=-99.1",size=4,color="black") +
         geom_text(data = est_est_label,label = "est=-53.9",size=4,color="black") +
         geom_text(data = fas_est_label,label = "est=-45.9",size=4,color="black") +
         geom_text(data = fra_est_label,label = "est=-70.3",size=4,color="black") +
         geom_text(data = hrv_est_label,label = "est=-27.7",size=4,color="black") +
         geom_text(data = jpn_est_label,label = "est=-26.9",size=4,color="black") +
         geom_text(data = kor_est_label,label = "est=-79.5",size=4,color="black") +
#          geom_text(data = nor_est_label,label = "est=-.68",size=4,color="black") +
         geom_text(data = por_est_label,label = "est=-29.1",size=4,color="black") +
         geom_text(data = spa_est_label,label = "est=18.8",size=4,color="black") +
#          geom_text(data = swe_est_label,label = "est=-43.4",size=4,color="black") +
         geom_text(data = zho_est_label,label = "est=-89.5",size=4,color="black")
         

ggsave("../figures/lexical_diversity_maxturn_eff.pdf", width = 11.7, height = 6.2)

\+ sample size

In [42]:
%%R

deu_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="German")
eng_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="English")
est_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Estonian")
fas_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Persian")
fra_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="French")
hrv_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Croatian")
jpn_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Japanese")
kor_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Korean")
nor_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Norwegian")
pol_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Polish")
por_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Portuguese")
spa_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Spanish")
swe_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Swedish")
zho_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Mandarin")

deu_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="German")
eng_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="English")
est_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Estonian")
fas_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Persian")
fra_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="French")
hrv_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Croatian")
jpn_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Japanese")
kor_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Korean")
nor_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Norwegian")
pol_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Polish")
por_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Portuguese")
spa_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Spanish")
swe_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Swedish")
zho_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Mandarin")

p <- p + geom_text(data = deu_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = eng_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = est_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = fas_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = fra_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = hrv_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = jpn_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = kor_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = nor_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = pol_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = por_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = spa_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = swe_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = zho_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = deu_sz_label,label = " = 39",size=4,color="black") +
         geom_text(data = eng_sz_label,label = " =1010",size=4,color="black") +
         geom_text(data = est_sz_label,label = " = 22",size=4,color="black") +
         geom_text(data = fas_sz_label,label = " = 12",size=4,color="black") +
         geom_text(data = fra_sz_label,label = " = 258",size=4,color="black") +
         geom_text(data = hrv_sz_label,label = " = 79",size=4,color="black") +
         geom_text(data = jpn_sz_label,label = " = 139",size=4,color="black") +
         geom_text(data = kor_sz_label,label = " = 37",size=4,color="black") +
         geom_text(data = nor_sz_label,label = " = 56",size=4,color="black") +
         geom_text(data = pol_sz_label,label = " = 1",size=4,color="black") +
         geom_text(data = por_sz_label,label = " = 24",size=4,color="black") +
         geom_text(data = spa_sz_label,label = " = 31",size=4,color="black") +
         geom_text(data = swe_sz_label,label = " = 16",size=4,color="black") +
         geom_text(data = zho_sz_label,label = " = 2",size=4,color="black")
         

ggsave("../figures/lexical_diversity_maxturn_eff_n.pdf", width = 11.7, height = 6.2)

----
#### Lexical diversity mixed models

In [8]:
# ara=maxturn_dat_inc_cg[['language','num_tokens','contingent','transcript_id','target_child_id']][maxturn_dat_inc_cg["language"]=="ara"] # no adult speech transcribed
deu=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="German"]
eng=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="English"]
est=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Estonian"]
fas=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Persian"]
fra=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="French"]
hrv=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Croatian"]
jpn=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Japanese"]
kor=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Korean"]
nor=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Norwegian"]
pol=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Polish"]
por=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Portuguese"]
spa=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Spanish"]
swe=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Swedish"]
zho=maxturn_lex_stats[['Language_name','sums','contingent','transcript_id','target_child_id']][maxturn_lex_stats["Language_name"]=="Mandarin"]

In [22]:
%%R

library("lme4")
library("emmeans")
library("lmerTest")
library("tidyverse")

effect_sizes <- read.csv("../data/lexdiv_effect_sizes.csv")

effect_sizes["max_turn_effect_size"] <- NA

In [23]:
%%R -i deu

lm2 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=deu, REML= FALSE)
emm2<-emmeans(lm2,pairwise~contingent)
pval<-summary(emm2$contrasts)$p.value
print(c(emm2$contrasts, pval))
print(p.adjust(pval, "holm", 14)) # create big vector of p-values and ajdust those
# summary(emmeans(lm2,"contingent",contr="pairwise"),infer=TRUE) #group means
# test(contrast(emmeans(lm2,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

deu_lname <- deu$Language_name[1]

deu_eff <- eff_size(emm2,sigma = sigma(lm2), edf = df.residual(lm2))

deu_eff <- summary(deu_eff)$effect.size

effect_sizes[effect_sizes$Language_name==deu_lname,"max_turn_effect_size"] <- deu_eff
effect_sizes

[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)    -46.6 8.28 39 -5.632  <.0001 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 1.67604e-06

[1] 2.346456e-05


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846                   NA
3       Estonian        -1.305378          -0.9327983                   NA
4        Persian        -2.931949          -1.8931652                   NA
5         French        -1.738904          -1.5925148                   NA
6       Croatian        -1.783383          -1.1007646                   NA
7       Japanese        -1.686991          -1.1232899                   NA
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [24]:
%%R -i eng

lm3 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=eng, REML= FALSE)
emm3<-emmeans(lm3,pairwise~contingent)
pval<-summary(emm3$contrasts)$p.value
print(c(emm3$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm3,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm3,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

eng_lname <- eng$Language_name[1]

eng_eff <- eff_size(emm3,sigma = sigma(lm3), edf = df.residual(lm3))

eng_eff <- summary(eng_eff)$effect.size

effect_sizes[effect_sizes$Language_name==eng_lname,"max_turn_effect_size"] <- eng_eff
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE  df t.ratio p.value
 contingent - (non-contingent)    -98.4 1.93 997 -51.021 <.0001 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 3.609764e-280

[1] 5.05367e-279


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983                   NA
4        Persian        -2.931949          -1.8931652                   NA
5         French        -1.738904          -1.5925148                   NA
6       Croatian        -1.783383          -1.1007646                   NA
7       Japanese        -1.686991          -1.1232899                   NA
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [25]:
%%R -i est

lm4 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=est, REML= FALSE)
emm4<-emmeans(lm4,pairwise~contingent)
pval<-summary(emm4$contrasts)$p.value
print(c(emm4$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm4,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm4,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

est_lname <- est$Language_name[1]

est_eff <- eff_size(emm4,sigma = sigma(lm4), edf = df.residual(lm4))

est_eff <- summary(est_eff)$effect.size

effect_sizes[effect_sizes$Language_name==est_lname,"max_turn_effect_size"] <- est_eff
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -53.8 13.2 23.1 -4.076  0.0005 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.0004639532

[1] 0.006495345


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652                   NA
5         French        -1.738904          -1.5925148                   NA
6       Croatian        -1.783383          -1.1007646                   NA
7       Japanese        -1.686991          -1.1232899                   NA
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [26]:
%%R -i fas

lm5 <- lmer(sums ~ contingent + (1|transcript_id),data=fas, REML= FALSE)
emm5<-emmeans(lm5,pairwise~contingent)
pval<-summary(emm5$contrasts)$p.value
print(c(emm5$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm5,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm5,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

fas_lname <- fas$Language_name[1]

fas_eff <- eff_size(emm5,sigma = sigma(lm5), edf = df.residual(lm5))

fas_eff <- summary(fas_eff)$effect.size

effect_sizes[effect_sizes$Language_name==fas_lname,"max_turn_effect_size"] <- fas_eff
effect_sizes

[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -45.7 11.2 13.1 -4.086  0.0013 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.001267753

[1] 0.01774854


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148                   NA
6       Croatian        -1.783383          -1.1007646                   NA
7       Japanese        -1.686991          -1.1232899                   NA
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [27]:
%%R -i fra

lm6 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=fra, REML= FALSE)
emm6<-emmeans(lm6,pairwise~contingent)
pval<-summary(emm6$contrasts)$p.value
print(c(emm6$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm6,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm6,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

fra_lname <- fra$Language_name[1]

fra_eff <- eff_size(emm6,sigma = sigma(lm6), edf = df.residual(lm6))

fra_eff <- summary(fra_eff)$effect.size

effect_sizes[effect_sizes$Language_name==fra_lname,"max_turn_effect_size"] <- fra_eff
effect_sizes

[[1]]
 contrast                      estimate   SE  df t.ratio p.value
 contingent - (non-contingent)    -70.1 3.08 251 -22.761 <.0001 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 5.2023e-63

[1] 7.28322e-62


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646                   NA
7       Japanese        -1.686991          -1.1232899                   NA
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [28]:
%%R -i hrv

lm7 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=hrv, REML= FALSE)
emm7<-emmeans(lm7,pairwise~contingent)
pval<-summary(emm7$contrasts)$p.value
print(c(emm7$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm7,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm7,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

hrv_lname <- hrv$Language_name[1]

hrv_eff <- eff_size(emm7,sigma = sigma(lm7), edf = df.residual(lm7))

hrv_eff <- summary(hrv_eff)$effect.size

effect_sizes[effect_sizes$Language_name==hrv_lname,"max_turn_effect_size"] <- hrv_eff
effect_sizes

[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -27.4 4.28 59.6 -6.406  <.0001 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 2.613052e-08

[1] 3.658273e-07


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646            -1.198342
7       Japanese        -1.686991          -1.1232899                   NA
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [29]:
%%R -i jpn

lm8 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=jpn, REML= FALSE)
emm8<-emmeans(lm8,pairwise~contingent)
pval<-summary(emm8$contrasts)$p.value
print(c(emm8$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm8,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm8,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

jpn_lname <- jpn$Language_name[1]

jpn_eff <- eff_size(emm8,sigma = sigma(lm8), edf = df.residual(lm8))

jpn_eff <- summary(jpn_eff)$effect.size

effect_sizes[effect_sizes$Language_name==jpn_lname,"max_turn_effect_size"] <- jpn_eff
effect_sizes

[[1]]
 contrast                      estimate   SE  df t.ratio p.value
 contingent - (non-contingent)    -27.5 2.91 140 -9.444  <.0001 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 1.103714e-16

[1] 1.545199e-15


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646            -1.198342
7       Japanese        -1.686991          -1.1232899            -1.136959
8         Korean        -2.152637          -1.5402032                   NA
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [30]:
%%R -i kor

lm9 <- lmer(sums ~ contingent + (1|transcript_id), data=kor, REML= FALSE)
emm9<-emmeans(lm9,pairwise~contingent)
pval<-summary(emm9$contrasts)$p.value
print(c(emm9$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm9,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm9,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

kor_lname <- kor$Language_name[1]

kor_eff <- eff_size(emm9,sigma = sigma(lm9), edf = df.residual(lm9))

kor_eff <- summary(kor_eff)$effect.size

effect_sizes[effect_sizes$Language_name==kor_lname,"max_turn_effect_size"] <- kor_eff
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)    -79.5 11.8 29 -6.749  <.0001 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 2.075041e-07

[1] 2.905058e-06


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646            -1.198342
7       Japanese        -1.686991          -1.1232899            -1.136959
8         Korean        -2.152637          -1.5402032            -1.836832
9      Norwegian               NA                  NA                   NA
10        Polish               NA                  NA                   NA
11    Portuguese        -1.405287                  NA                   NA
12       Spanish               NA                  NA                   NA
13       Swedish         

In [31]:
%%R -i nor

lm10 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=nor, REML= FALSE)
emm10<-emmeans(lm10,pairwise~contingent)
pval<-summary(emm10$contrasts)$p.value
print(c(emm10$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm10,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm10,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

nor_lname <- nor$Language_name[1]

effect_sizes[effect_sizes$Language_name==nor_lname,"max_turn_effect_size"] <- NaN
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -16.3 7.15 43.5 -2.283  0.0274 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.02737409

[1] 0.3832373
   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646            -1.198342
7       Japanese        -1.686991          -1.1232899            -1.136959
8         Korean        -2.152637          -1.5402032            -1.836832
9      Norwegian               NA                  NA                  NaN
10        Polish               N

In [32]:
%%R -i pol

# simple linear model (no maxturnom effects, because only 1 transcript from 1 sub)

#

lm11 <- lm(sums ~ contingent ,data=pol, REML= FALSE)
emm11<-emmeans(lm11,pairwise~contingent)
pval<-summary(emm11$contrasts)$p.value
print(c(emm11$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm11,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm11,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

pol_lname <- pol$Language_name[1]

effect_sizes[effect_sizes$Language_name==pol_lname,"max_turn_effect_size"] <- NaN
effect_sizes

[[1]]
 contrast                      estimate  SE df t.ratio p.value
 contingent - (non-contingent)      -10 NaN  0 NaN     NaN    


[[2]]
[1] NaN

[1] NaN
   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646            -1.198342
7       Japanese        -1.686991          -1.1232899            -1.136959
8         Korean        -2.152637          -1.5402032            -1.836832
9      Norwegian               NA                  NA                  NaN
10        Polish               NA                  NA                  NaN
11    Portuguese  

In [33]:
%%R -i por

lm12 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=por, REML= FALSE)
emm12<-emmeans(lm12,pairwise~contingent)
pval<-summary(emm12$contrasts)$p.value
print(c(emm12$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm12,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm12,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

por_lname <- por$Language_name[1]

por_eff <- eff_size(emm12,sigma = sigma(lm12), edf = df.residual(lm12))

por_eff <- summary(por_eff)$effect.size

effect_sizes[effect_sizes$Language_name==por_lname,"max_turn_effect_size"] <- por_eff
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -28.3 7.59 24.1 -3.721  0.0011 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.001059215

[1] 0.01482901


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828            -1.309425
2        English        -2.469682          -2.2049846            -2.300346
3       Estonian        -1.305378          -0.9327983            -1.257935
4        Persian        -2.931949          -1.8931652            -1.742468
5         French        -1.738904          -1.5925148            -2.061280
6       Croatian        -1.783383          -1.1007646            -1.198342
7       Japanese        -1.686991          -1.1232899            -1.136959
8         Korean        -2.152637          -1.5402032            -1.836832
9      Norwegian               NA                  NA                  NaN
10        Polish               NA                  NA                  NaN
11    Portuguese        -1.405287                  NA            -1.122066
12       Spanish               NA                  NA                   NA
13       Swedish         

In [34]:
%%R -i spa

lm13 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=spa, REML= FALSE)
emm13<-emmeans(lm13,pairwise~contingent)
pval<-summary(emm13$contrasts)$p.value
print(c(emm13$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm13,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm13,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

spa_lname <- spa$Language_name[1]

spa_eff <- eff_size(emm13,sigma = sigma(lm13), edf = df.residual(lm13))

spa_eff <- summary(spa_eff)$effect.size

effect_sizes[effect_sizes$Language_name==spa_lname,"max_turn_effect_size"] <- spa_eff
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)     19.1 5.53 32 3.452   0.0016 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.001582076

[1] 0.02214906


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828           -1.3094253
2        English        -2.469682          -2.2049846           -2.3003460
3       Estonian        -1.305378          -0.9327983           -1.2579348
4        Persian        -2.931949          -1.8931652           -1.7424681
5         French        -1.738904          -1.5925148           -2.0612800
6       Croatian        -1.783383          -1.1007646           -1.1983419
7       Japanese        -1.686991          -1.1232899           -1.1369590
8         Korean        -2.152637          -1.5402032           -1.8368325
9      Norwegian               NA                  NA                  NaN
10        Polish               NA                  NA                  NaN
11    Portuguese        -1.405287                  NA           -1.1220664
12       Spanish               NA                  NA            0.8914125
13       Swedish         

In [35]:
%%R -i swe

lm14 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=swe, REML= FALSE)
emm14<-emmeans(lm14,pairwise~contingent)
pval<-summary(emm14$contrasts)$p.value
print(c(emm14$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm14,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm14,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

swe_lname <- swe$Language_name[1]

effect_sizes[effect_sizes$Language_name==swe_lname,"max_turn_effect_size"] <- NaN
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -25.1 14.7 17.1 -1.703  0.1067 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.1067129

[1] 1
   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828           -1.3094253
2        English        -2.469682          -2.2049846           -2.3003460
3       Estonian        -1.305378          -0.9327983           -1.2579348
4        Persian        -2.931949          -1.8931652           -1.7424681
5         French        -1.738904          -1.5925148           -2.0612800
6       Croatian        -1.783383          -1.1007646           -1.1983419
7       Japanese        -1.686991          -1.1232899           -1.1369590
8         Korean        -2.152637          -1.5402032           -1.8368325
9      Norwegian               NA                  NA                  NaN
10        Polish               NA        

In [36]:
%%R -i zho

lm15 <- lmer(sums ~ contingent + (1|transcript_id),data=zho, REML= FALSE)
emm15<-emmeans(lm15,pairwise~contingent)
pval<-summary(emm15$contrasts)$p.value
print(c(emm15$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm15,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm15,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

zho_lname <- zho$Language_name[1]

zho_eff <- eff_size(emm15,sigma = sigma(lm15), edf = df.residual(lm15))

zho_eff <- summary(zho_eff)$effect.size

effect_sizes[effect_sizes$Language_name==zho_lname,"max_turn_effect_size"] <- zho_eff
effect_sizes

[[1]]
 contrast                      estimate  SE df t.ratio p.value
 contingent - (non-contingent)    -89.5 7.5  4 -11.933 0.0003 

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.0002825224

[1] 0.003955313


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name rand_effect_size max_voc_effect_size max_turn_effect_size
1         German        -1.823008          -1.1826828           -1.3094253
2        English        -2.469682          -2.2049846           -2.3003460
3       Estonian        -1.305378          -0.9327983           -1.2579348
4        Persian        -2.931949          -1.8931652           -1.7424681
5         French        -1.738904          -1.5925148           -2.0612800
6       Croatian        -1.783383          -1.1007646           -1.1983419
7       Japanese        -1.686991          -1.1232899           -1.1369590
8         Korean        -2.152637          -1.5402032           -1.8368325
9      Norwegian               NA                  NA                  NaN
10        Polish               NA                  NA                  NaN
11    Portuguese        -1.405287                  NA           -1.1220664
12       Spanish               NA                  NA            0.8914125
13       Swedish         

In [37]:
%%R
write.csv(x=effect_sizes,'../data/lexdiv_effect_sizes.csv', row.names = FALSE)